In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Project import Project

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd

from Scripts.data.DataLoading import get_image_dataloaders, get_image_dataset
from Scripts.data.SSVEPDataset import SSVEPDataset
from Scripts.data.SSVEPDataloader import SSVEPDataloader
from Scripts.neuralnets.NNPreinstalledModelSelection import *
from Scripts.neuralnets.NNTrainingUtils import train, kfold_train

In [4]:
ngpu = 1; device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

***

Training with simple cross validation.

In [57]:
preprocessing = 'cca'
signal_length = '512'
fig_type = 'rp'
dataloaders, dataset_sizes = get_image_dataloaders(Project, preprocessing, signal_length, fig_type, batch_size = 8)

In [58]:
model_type = 'googlenet'; model_size = ''; num_classes = 5; model_name = model_type + model_size
model = model_selection(model_name, num_classes, pretrained = True).to(device)

In [59]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)

In [60]:
model, stats = train(Project, model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs = 20)

***

Training with K-Fold cross validation.

In [21]:
preprocessing = 'cca'
signal_length = '512'
fig_type = 'gasf'
dataset = get_image_dataset(Project, preprocessing, signal_length, fig_type, image_size = 300)
print(len(dataset))

2730


In [22]:
model_type = 'resnet'; model_size = '152'; num_classes = 5; model_name = model_type + model_size
model = model_selection(model_name, num_classes).to(device)

In [23]:
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.5)

In [24]:
model, stats = kfold_train(Project, model, dataset, criterion, optimizer, num_epochs = 20, num_folds = 5, batch_size = 4)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\olipp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-6833e8da6d57>", line 1, in <module>
    model, stats = kfold_train(Project, model, dataset, criterion, optimizer, num_epochs = 20, num_folds = 5, batch_size = 4)
  File "C:\Users\olipp\Documents\GitHub\ICDeepLearning\Scripts\neuralnets\NNTrainingUtils.py", line 210, in kfold_train
    model, stats = train(project, model, dataloaders, dataset_size, criterion, optimizer, scheduler, num_epochs, logger = logger, kfold = True)
  File "C:\Users\olipp\Documents\GitHub\ICDeepLearning\Scripts\neuralnets\NNTrainingUtils.py", line 136, in train
    for inputs, labels in dataloaders[phase]:
  File "C:\Users\olipp\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 363, in __next__
    data = self._next_data()
  File "C:\Users\olipp\anaconda3\lib\site-packages\torc

TypeError: object of type 'NoneType' has no len()

***

Training with K-Fold cross validation and data preprocessed uniquely with CCA.

In [36]:
preprocessing = 'cca'
signal_length = '128'
data = pd.read_csv((Project.output_dir / ('SSVEPDataset_'+signal_length+'.csv')).__str__(), dtype = 'str')
dataset = SSVEPDataset(Project, preprocessing, data, signal_length)

In [37]:
batch_size = 64
dataloaders, dataset_sizes = SSVEPDataloader(dataset, batch_size)
print(dataset_sizes)

{'train': 8716, 'val': 2178}


In [38]:
model_type = 'resnet'; model_size = '50'; num_classes = 5; model_name = model_type + model_size
model = model_selection(model_name, num_classes)
model.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
model = model.to(device)

In [39]:
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [40]:
model, stats = kfold_train(Project, model, dataset, criterion, optimizer, num_epochs = 50, num_folds = 5, batch_size = batch_size)